In [1]:
from newspaper import Article
import pandas as pd
import re
import torch

In [2]:
def clean(x):
    return x.replace("(","").replace(")","").replace("{","").replace("}","").replace("[","").replace("]","").replace('"',"").replace("'","").replace("=","")
def _get_ngrams(n, text):
    """Calcualtes n-grams.

    Args:
      n: which n-grams to calculate
      text: An array of tokens

    Returns:
      A set of n-grams
    """
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set


def _get_word_ngrams(n, sentences):
    """Calculates word n-grams for multiple sentences.
    """
    assert len(sentences) > 0
    assert n > 0

    # words = _split_into_words(sentences)

    words = sum(sentences, [])
    # words = [w for w in words if w not in stopwords]
    return _get_ngrams(n, words)

In [3]:
url1="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=015&aid=0004435156&date=20201021&type=1&rankingSeq=1&rankingSectionId=104"
url2="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=296&aid=0000046738&date=20201011&type=1&rankingSeq=2&rankingSectionId=103"
url3="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=009&aid=0004678970&date=20201021&type=1&rankingSeq=5&rankingSectionId=103"
url4="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=025&aid=0003045159&date=20201021&type=1&rankingSeq=1&rankingSectionId=105"
url5="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=138&aid=0002093413&date=20201021&type=1&rankingSeq=2&rankingSectionId=105"
url6="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=056&aid=0010920303&date=20201021&type=1&rankingSeq=3&rankingSectionId=105"

In [4]:
article=Article(url1, language='ko')
article.download()
article.parse()
text1=article.text
article=Article(url2, language='ko')
article.download()
article.parse()
text2=article.text
article=Article(url3, language='ko')
article.download()
article.parse()
text3=article.text
article=Article(url4, language='ko')
article.download()
article.parse()
text4=article.text
article=Article(url5, language='ko')
article.download()
article.parse()
text5=article.text
article=Article(url6, language='ko')
article.download()
article.parse()
text6=article.text

In [5]:
tgt_text1="그런데 왓츠앱의 경우 암호화 처리를 하기 때문에 왓츠앱을 통해 주고받은 대화 내용을 은행에서 확인하기가 쉽지 않다는 점이 문제가 돼 왔다. 그러나 은행 직원들은 왓츠앱을 통해 주변 사람들과 편하게 대화를 나눌 수 있기 때문에 회사 방침에도 불구하고 사용을 전면 중단하지는 않고 있다. 모간스탠리가 이번에 해고한 킹 대표 등 두 명 모두 왓츠앱을 통해 위법행위를 저지르지 않은 것으로 드러났다."
tgt_text2="건강수명을 누리려면 먼저 치매, 만성질환, 암 등 치료가 어렵고 투병기간이 긴 질병부터 예방해야 한다. 근육을 저축하고 키우기 위해서는 근력운동과 단백질 섭취, 비타민 D 섭취 등을 동시에 하는 것이 가장 좋다. 젊을 때부터 근력운동을 통해 근육의 힘을 많이 키워놓으면 크게 도움이 된다."
tgt_text3="롯데그룹이 갈수록 치열해지는 온·오프라인 유통시장 경쟁에서 승리하기 위해 본격적인 빅데이터 경영에 나선다. 데이터를 무기로 시장을 잠식하는 네이버와 카카오 등 IT공룡들의 공세에 맞서 강희태 유통BU장·롯데쇼핑 부회장 직속의 빅데이터 조직을 발족, 그간 각 계열사별로 따로 관리하던 소비데이터를 그룹 차원에서 한데 모아 분석하고 이에 맞는 맞춤형 쇼핑서비스를 선보인다는 목표다. 21일 롯데에 따르면 강 부회장이 이끄는 롯데 유통BU는 지난 1일 강 부회장 직속의 데이터 거버넌스 태스크포스를 출범하고, TF장이자 CDO로 윤영선 롯데정보통신 상무를 임명했다."
tgt_text4="8년간 의식장애로 인한 마비로 침대에 누워있거나 휠체어에 묶여 있던 네덜란드 30대 남성이 수면제 복용 20분 만에 정상능력을 회복해 다시 걷고 말하고, 먹을 수 있게 됐다. 한편 앞서도 혼수상태에 있던 환자가 수면제 복용 뒤 일시적으로 정상 능력을 회복했다는 연구결과가 있었다. 연구팀은 리처드 치료를 계기로 수면제를 활용해 뇌손상이나 혼수상태에 있는 환자를 영구적인 정상 상태로 회복시키는 방안을 연구하고 있다."
tgt_text5="예상보다 글로벌 가입자가 적게 늘었다. 대신 한국과 일본이 체면치레를 해줬다. 넷플릭스가 이번 분기 국내에서 상당한 수익을 벌고 갈 것으로 예상되면서 국회를 중심으로 한국시장 무임승차 논란이 다시 한번 불거질 전망이다."
tgt_text6="미국의 소행성 탐사선 '오시리스-렉스'가 20일 소행성 '101955 베누' 표면에 접지해 암석 표본을 채집하는 데 성공했습니다. 이번에 채취된 베누의 샘플은 2023년쯤 받아볼 수 있을 전망입니다. 나사는 이를 통해 베누의 구성 성분과 기원, 형성과정 등을 살피고 수분의 존재 여부 등도 조사하게 됩니다."

In [6]:
df=pd.DataFrame({"body":[text1,text2,text3,text4,text5,text6],"tgt":[tgt_text1,tgt_text2,tgt_text3,tgt_text4,tgt_text5,tgt_text6]})
df

,body,tgt
0,사진=연합뉴스 사진=연합뉴스\n\n미국 월스트리트에서 거액 연봉을 받던 트레이더들이...,그런데 왓츠앱의 경우 암호화 처리를 하기 때문에 왓츠앱을 통해 주고받은 대화 내용을...
1,"[사진=게티이미지뱅크] [사진=게티이미지뱅크]\n\n요즘은 ""오래사세요""(장수)보다...","건강수명을 누리려면 먼저 치매, 만성질환, 암 등 치료가 어렵고 투병기간이 긴 질병..."
2,롯데그룹이 갈수록 치열해지는 온·오프라인 유통시장 경쟁에서 승리하기 위해 본격적인 ...,롯데그룹이 갈수록 치열해지는 온·오프라인 유통시장 경쟁에서 승리하기 위해 본격적인 ...
3,졸피뎀. 연합뉴스 졸피뎀. 연합뉴스\n\n8년간 의식장애로 인한 마비로 침대에 누워...,8년간 의식장애로 인한 마비로 침대에 누워있거나 휠체어에 묶여 있던 네덜란드 30대...
4,[디지털데일리 권하영기자] 넷플릭스가 기대이하 성적을 냈다. 예상보다 글로벌 가입자...,예상보다 글로벌 가입자가 적게 늘었다. 대신 한국과 일본이 체면치레를 해줬다. 넷플...
5,사진출처: NASA 사진출처: NASA\n\n사진출처: NASA 사진출처: NASA...,미국의 소행성 탐사선 '오시리스-렉스'가 20일 소행성 '101955 베누' 표면에...


In [9]:
df[:5].to_csv("train_data.csv",index=False)
df[5:].to_csv("test_data.csv",index=False)

In [10]:
from konlpy.tag import Kkma
def load_data(row):
    kkma=Kkma()
    article=row["body"]
    abstract=row["tgt"]
#     article=article.text
#     abstract=tgt_text
    src_txt=kkma.sentences(article)
    tgt_txt=kkma.sentences(abstract)

    source=[kkma.morphs(clean(sen)) for sen in src_txt]
    tgt=[kkma.morphs(clean(sen)) for sen in tgt_txt]
    return source,tgt
def _format_to_lines(f):
    source, tgt = load_data(f)
    return {'src': source, 'tgt': tgt}

def format_to_lines():
    for corpus_type in ['test', 'train']:
        df=pd.read_csv(corpus_type+"_data.csv")
        df.apply(_format_to_lines,axis=1).to_json("cnndm_sample."+corpus_type+".0"+".json",orient="records")
format_to_lines()